<a href="https://colab.research.google.com/gist/beavis28/3661d9d697141ebd6912d7fd9bed5471/echnical-analysis-ma-bb-rsi-macd-anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install alpha_vantage
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from datetime import date
from dateutil.relativedelta import relativedelta
from dateutil import parser
import numpy as np
import math
from alpha_vantage.techindicators import TechIndicators

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [24, 8]
API_KEY = '0A5DAC7F3S5UWRJZ'

In [3]:
symbol = "SNAP"

In [4]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
ti = TechIndicators(key=API_KEY, output_format='pandas')
ts = TimeSeries(key=API_KEY, output_format='pandas')
data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
data['Price_Up_From_PreviousDay'] = np.where(data['4. close'] > data['4. close'].shift(-1), 1, 0)
data['Today_Price_Up'] = np.where(data['4. close'] > data['1. open'], 1, 0)
data['Tomorrow_Price_Up'] = np.where(data['4. close'].shift(1) > data['4. close'], 1, 0)
data['Percentage'] = (data['4. close'] - data['4. close'].shift(-1)) / data['4. close'].shift(-1)
data['SomeDaysAfterClosingPrice'] = data['4. close'].shift(10) # X days later price
data

,1. open,2. high,3. low,4. close,5. volume,Price_Up_From_PreviousDay,Today_Price_Up,Tomorrow_Price_Up,Percentage,SomeDaysAfterClosingPrice
date,,,,,,,,,,
2021-11-11,53.460,54.5900,53.20,53.29,13268265.0,1,0,0,0.007753,NaN
2021-11-10,53.910,54.1699,52.02,52.88,14812395.0,0,0,1,-0.035389,NaN
2021-11-09,55.295,55.6500,54.20,54.82,12393362.0,0,0,0,-0.001457,NaN
2021-11-08,53.360,55.1300,52.85,54.90,20154579.0,1,1,0,0.032537,NaN
2021-11-05,52.150,53.5700,52.07,53.17,17458733.0,1,1,1,0.017413,NaN
...,...,...,...,...,...,...,...,...,...,...
2017-03-08,22.030,23.4300,21.31,22.81,49834423.0,1,1,0,0.063899,21.82
2017-03-07,22.210,22.5000,20.64,21.44,71899652.0,0,0,1,-0.098023,20.38
2017-03-06,28.170,28.2500,23.77,23.77,72938848.0,0,0,0,-0.122554,19.93


In [5]:
# Moving Average 5 and 20
dataSMA_short, meta_data_sma_short = ti.get_sma(symbol=symbol, time_period=5)
dataSMA_short.columns = ['SMA_short']
data = data.merge(dataSMA_short, left_index=True, right_index=True)

dataSMA_long, meta_data_sma_long = ti.get_sma(symbol=symbol, time_period=20)
dataSMA_long.columns = ['SMA_long']
data = data.merge(dataSMA_long, left_index=True, right_index=True)

data['SMATrend'] = np.where(data['SMA_short'] > data['SMA_long'], 1, 0) # Up trend is 1,, Down Trend is 0
data['GoldenCrossHappened'] = np.where(data['SMATrend'] > data['SMATrend'].shift(-1), 1, 0)
#pd.set_option('display.max_rows', None)

# theory 1
# we should see price up after seeing golden cross between SMA 5 and SMA20.
# this is giving as some days later price as example
turningPointChange = 0
priceActuallyUp = 0
for index, row in data.iterrows():
    if row['GoldenCrossHappened'] == 1:
      turningPointChange += 1
      if row['4. close'] < row['SomeDaysAfterClosingPrice']:
        priceActuallyUp += 1

print("Golden Cross Point:  " + str(turningPointChange))
print("Actual Price Up:  " + str(priceActuallyUp))
print("Percentage:  " + str((priceActuallyUp/turningPointChange)*100) + "%")

def interactive_plot(df, title):
  fig = px.line(title = title)
  
  # Loop through each stock (while ignoring time columns with index 0)
  for i in df.columns[:]:
    if i == "4. close" or i == "SMA_short" or i == "SMA_long":
      fig.add_scatter(x = df.index, y = df[i], name = i) # add a new Scatter trace

  fig.show()

interactive_plot(data, 'Prices')

Golden Cross Point:  31
Actual Price Up:  17
Percentage:  54.83870967741935%


In [6]:
#BB
dataBB, meta_data_bb = ti.get_bbands(symbol=symbol)
data = data.merge(dataBB, left_index=True, right_index=True)
data['PriceBelowLowerBB'] = np.where(data['Real Lower Band'] > data['4. close'], 1, 0)
data['TouchDownOnLowerBBHappened'] = np.where(data['PriceBelowLowerBB'] > data['PriceBelowLowerBB'].shift(-1), 1, 0)
# theory 2
# we should see price up after reaching lower BB.
# this is giving as some days later price as example
turningPointChange = 0
priceActuallyUp = 0
for index, row in data.iterrows():
    if row['TouchDownOnLowerBBHappened'] == 1:
      turningPointChange += 1
      if row['4. close'] < row['SomeDaysAfterClosingPrice']:
        priceActuallyUp += 1

print("LowerBBTouched Point:  " + str(turningPointChange))
print("Actual Price Up:  " + str(priceActuallyUp))
print("Percentage:  " + str((priceActuallyUp/turningPointChange)*100) + "%")

def interactive_plot(df, title):
  fig = px.line(title = title)
  
  # Loop through each stock (while ignoring time columns with index 0)
  for i in df.columns[:]:
    if i == "4. close" or i == "Real Middle Band" or i == "Real Upper Band" or i == "Real Lower Band":
      fig.add_scatter(x = df.index, y = df[i], name = i) # add a new Scatter trace

  fig.show()

interactive_plot(data, 'Prices')

LowerBBTouched Point:  38
Actual Price Up:  16
Percentage:  42.10526315789473%


In [7]:
# RSI
dataRSI, meta_data_rsi = ti.get_rsi(symbol=symbol)
data = data.merge(dataRSI, left_index=True, right_index=True)
data['RSITouched30'] = np.where(data['RSI'] < 30, 1, 0)
data['TouchDownRSI30Happened'] = np.where(data['RSITouched30'] > data['RSITouched30'].shift(-1), 1, 0)
# theory 3
# we should see price up after reaching RSI30.
# this is giving as some days later price as example
turningPointChange = 0
priceActuallyUp = 0
for index, row in data.iterrows():
    if row['TouchDownRSI30Happened'] == 1:
      turningPointChange += 1
      if row['4. close'] < row['SomeDaysAfterClosingPrice']:
        priceActuallyUp += 1

print("Price Touch RSI30 Point:  " + str(turningPointChange))
print("Actual Price Up:  " + str(priceActuallyUp))
print("Percentage:  " + str((priceActuallyUp/turningPointChange)*100) + "%")

Price Touch RSI30 Point:  10
Actual Price Up:  4
Percentage:  40.0%


In [8]:
#MACD
dataMACD, meta_data_macd = ti.get_macd(symbol=symbol, interval='daily')
data = data.merge(dataMACD, left_index=True, right_index=True)
data['MACDOverSignal'] = np.where(data['MACD'] > data['MACD_Signal'], 1, 0)
data['MACDHistUp'] = np.where(data['MACD_Hist'] > data['MACD_Hist'].shift(-2), 1, 0)
data['MACDOverSignalTrendHappened'] = np.where(data['MACDOverSignal'] > data['MACDOverSignal'].shift(-1), 1, 0)
data['MACDHistUpTrendHappened'] = np.where(data['MACDHistUp'] > data['MACDHistUp'].shift(-1), 1, 0)
data['MACDUpTrendHappened'] = np.where((data['MACDOverSignalTrendHappened'] == 1) & (data['MACDHistUpTrendHappened'] == 1), 1, 0)
# theory 4
# we should see price up after reaching MACD Trend is up.
# this is giving as some days later price as example
turningPointChange = 0
priceActuallyUp = 0
for index, row in data.iterrows():
    if row['MACDUpTrendHappened'] == 1:
      turningPointChange += 1
      if row['4. close'] < row['SomeDaysAfterClosingPrice']:
        priceActuallyUp += 1

print("MACD Uptrend Point:  " + str(turningPointChange))
print("Actual Price Up:  " + str(priceActuallyUp))
print("Percentage:  " + str((priceActuallyUp/turningPointChange)*100) + "%")

MACD Uptrend Point:  4
Actual Price Up:  4
Percentage:  100.0%


In [9]:
#Calendar data
# which week of day get higher price than previous day
data = data.sort_index(ascending=1)
count = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0,'Friday': 0}
for index, row in data.iterrows():
  if index.day_name() == 'Monday' and int(row['Price_Up_From_PreviousDay']) == 1:
    count['Monday'] += 1
  elif index.day_name() == 'Tuesday' and int(row['Price_Up_From_PreviousDay']) == 1:
    count['Tuesday'] += 1
  elif index.day_name() == 'Wednesday' and int(row['Price_Up_From_PreviousDay']) == 1:
    count['Wednesday'] += 1
  elif index.day_name() == 'Thursday' and int(row['Price_Up_From_PreviousDay']) == 1:
    count['Thursday'] += 1  
  elif index.day_name() == 'Friday' and int(row['Price_Up_From_PreviousDay']) == 1:
    count['Friday'] += 1 

count     

{'Monday': 98,
 'Tuesday': 112,
 'Wednesday': 120,
 'Thursday': 122,
 'Friday': 126}

In [10]:
# Check which day of week get previous day and next day also got up
data = data.sort_index(ascending=1)
series_count = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0,'Friday': 0}
for index, row in data.iterrows():
  if index.day_name() == 'Monday' and int(row['Price_Up_From_PreviousDay']) == 1 and int(row['Tomorrow_Price_Up']) == 1:
    series_count['Monday'] += 1
  elif index.day_name() == 'Tuesday' and int(row['Price_Up_From_PreviousDay']) == 1 and int(row['Tomorrow_Price_Up']) == 1:
    series_count['Tuesday'] += 1
  elif index.day_name() == 'Wednesday' and int(row['Price_Up_From_PreviousDay']) == 1 and int(row['Tomorrow_Price_Up']) == 1:
    series_count['Wednesday'] += 1
  elif index.day_name() == 'Thursday' and int(row['Price_Up_From_PreviousDay']) == 1 and int(row['Tomorrow_Price_Up']) == 1:
    series_count['Thursday'] += 1  
  elif index.day_name() == 'Friday' and int(row['Price_Up_From_PreviousDay']) == 1 and int(row['Tomorrow_Price_Up']) == 1:
    series_count['Friday'] += 1  
series_count    

{'Monday': 38, 'Tuesday': 54, 'Wednesday': 63, 'Thursday': 66, 'Friday': 63}

In [11]:
# which week of day get higher average raise
percentageUp = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0,'Friday': 0}
total = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0,'Friday': 0}
for index, row in data.iterrows():
  if not math.isnan(row['Percentage']):
    if index.day_name() == 'Monday':
      percentageUp['Monday'] += row['Percentage']
      total['Monday'] += 1
    elif index.day_name() == 'Tuesday':
      percentageUp['Tuesday'] += row['Percentage']
      total['Tuesday'] += 1
    elif index.day_name() == 'Wednesday':
      percentageUp['Wednesday'] += row['Percentage']
      total['Wednesday'] += 1
    elif index.day_name() == 'Thursday':
      percentageUp['Thursday'] += row['Percentage']  
      total['Thursday'] += 1
    elif index.day_name() == 'Friday':
      percentageUp['Friday'] += row['Percentage'] 
      total['Friday'] += 1

percentageUp['Monday'] = percentageUp['Monday']/ total['Monday']*100
percentageUp['Tuesday'] = percentageUp['Tuesday']/ total['Tuesday']*100
percentageUp['Wednesday'] = percentageUp['Wednesday']/ total['Wednesday']*100
percentageUp['Thursday'] = percentageUp['Thursday']/ total['Thursday']*100
percentageUp['Friday'] = percentageUp['Friday']/ total['Friday']*100
percentageUp

{'Monday': -0.34080808168200905,
 'Tuesday': 0.0553467085300541,
 'Wednesday': 0.3127844979563143,
 'Thursday': 0.25601952164086095,
 'Friday': 0.5203673214686121}

In [12]:
# which week of day get more than 3% raise

total = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0,'Friday': 0}
for index, row in data.iterrows():
  if not math.isnan(row['Percentage']) and row['Percentage'] > 0.03:
    if index.day_name() == 'Monday':

      total['Monday'] += 1
    elif index.day_name() == 'Tuesday':

      total['Tuesday'] += 1
    elif index.day_name() == 'Wednesday':

      total['Wednesday'] += 1
    elif index.day_name() == 'Thursday':
 
      total['Thursday'] += 1
    elif index.day_name() == 'Friday':

      total['Friday'] += 1

total

{'Monday': 30, 'Tuesday': 31, 'Wednesday': 37, 'Thursday': 42, 'Friday': 46}

In [13]:
# historically how many days in a row they got price-up

data = data.sort_index(ascending=1)

days = 0
max_days = 0
percentage= 0
maxPercentage = 0
recorded_index = ""
recorded_index_row = ""
for index, row in data.iterrows():
  if not math.isnan(row['Percentage']):
    if row['Price_Up_From_PreviousDay'] == 1:
      days+= 1
      if days > max_days:
        max_days = days
        recorded_index_row = index

      percentage = row['Percentage']
      if percentage > maxPercentage:
        maxPercentage = percentage
        recorded_index = index
    else:
      #initialize
      days = 0   
      percentage = 0

print(recorded_index)
print("how much percentage they got up " + str(maxPercentage))
print(recorded_index_row)
print("how many days in row got up " + str(max_days))

2018-02-07 00:00:00
how much percentage they got up 0.4758179231863442
2020-06-23 00:00:00
how many days in row got up 8


In [14]:
# historically how many days in a row they got price-down

data = data.sort_index(ascending=1)

days = 0
max_days = 0
percentage= 0
maxPercentage = 0
recorded_index = ""
recorded_index_row = ""
for index, row in data.iterrows():
  if not math.isnan(row['Percentage']):
    if row['Price_Up_From_PreviousDay'] == 0:
      days+= 1
      if days > max_days:
        max_days = days
        recorded_index_row = index

      percentage = row['Percentage']*-1
      if percentage > maxPercentage:
        maxPercentage = percentage
        recorded_index = index
    else:

      #initialize
      days = 0   
      percentage = 0


print(recorded_index)
print("how much percentage they got down " + str(maxPercentage))
print(recorded_index_row)
print("how many days in row got down " + str(max_days))

2021-10-22 00:00:00
how much percentage they got down 0.2658767141525762
2017-07-12 00:00:00
how many days in row got down 8
